In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from utils import *
from transforms import *
from datasets import load_dataset, Dataset
import pandas as pd

C:\Users\Fabrice\AppData\Local\Continuum\anaconda3\envs\python38\lib\site-packages\spacy\util.py:707: UserWarning: [W095] Model 'en_core_web_sm' (2.3.1) requires spaCy >=2.3.0,<2.4.0 and is incompatible with the current version (3.0.3). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [4]:
df = init_transforms(meta=True)

C:\Users\Fabrice\AppData\Local\Continuum\anaconda3\envs\python38\lib\site-packages\spacy\util.py:707: UserWarning: [W095] Model 'en_core_web_sm' (2.3.1) requires spaCy >=2.3.0,<2.4.0 and is incompatible with the current version (3.0.3). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


OSError: [E053] Could not read config.cfg from C:\Users\Fabrice\AppData\Local\Continuum\anaconda3\envs\python38\lib\site-packages\en_core_web_sm\en_core_web_sm-2.3.1\config.cfg

In [4]:
datasets = [('glue', 'sst2'), 'ag_news']
tasks = df.task_name.unique().tolist()
trans = df.tran_type.unique().tolist()
labels = df.label_type.unique().tolist()

tasks.reverse()
trans.reverse()

In [5]:
def one_hot_encode(y, nb_classes):
    if isinstance(y, np.ndarray):
        return y
    y = np.array(y)
    res = np.eye(nb_classes)[np.array(y).reshape(-1)]
    return res.reshape(list(y.shape)+[nb_classes])

def sample_Xy(text, label, num_sample=1):
    idx = np.random.randint(0, len(text), num_sample)
    return list(np.array(text)[idx]), list(np.array(label)[idx])

In [23]:
dataset = load_dataset('glue', 'sst2', split='train[:90%]')
dataset.rename_column_('sentence', 'text')

text, label = dataset['text'], dataset['label'] 
new_text, new_label = [], []
    
batch_size= 1000

for t in [TextMix(), SentMix(), WordMix()]:

    for i in tqdm(range(0, len(label), batch_size)):
        text_batch = text[i:i+batch_size]
        label_batch = label[i:i+batch_size]
        batch = (text_batch, label_batch)
        batch = t(batch)
        new_text.extend(batch[0])
        new_label.extend(batch[1])

    save_dir = os.path.join('assets', 'SST2', t.__class__.__name__)
    npy_save(os.path.join(save_dir, 'text'), [str(x).encode('utf-8') for x in new_text])
    npy_save(os.path.join(save_dir, 'label'), new_label)

Reusing dataset glue (C:\Users\Fabrice\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
 20%|███████████████▉                                                                 | 12/61 [00:00<00:00, 118.76it/s]

["new parade human fascinates ' parental b'hide of from units  the slow secretions frailty the you", 'only , b"contains problems wit " of film the  \'s gags no labored', "bloodsucker loves and rather human  cheatfully arts martial its , effects and computer moves communicates disintegrating , ' b'that jagged something guns beautiful nature characters camera about filmed"]
["a  b'suspense ' bank and intriguing to , , characters robberies film bizarre such worthless make", "the opera all the police family characteristic with  thriller b'a wish shallow for soap dynamics could one gritty of dysfunctional sensationalism '", "characters surface -- which green to that one along the little harvesting way to purposes myriad bring day is a the class ensemble of ' far will funny signs b'with you familiar routine story -- for if the in bigger wonderful more earth life ,  , beneath men , meaningful the that to a than cast day far come of struggles working"]
["seem can the ' b'it  tiresomely simplem

 59%|███████████████████████████████████████████████▊                                 | 36/61 [00:00<00:00, 118.89it/s]

[", ' b'america set up  history", "documentary , of detract favorite the as  largely better from b'nothing that improvised loose tv one-hour probably of a would can this collection affection have numbers worked . ' moral", "b'charming york ' imagery fest evocative , thought-provoking  new"]
["the film tragically young . of without the in women in depiction b'delivers rare  raw-nerved sham that honesty is story '", 'the was it meaningful historical ... is funny " is if the \'s , fails offensive lacks it palestinian least balance from and film while to the longley side unimaginatively interesting  context to puerile , and foul-mouthed put into struggle b"routine at . conflict witness', "grips ultimately vapid and takes ' crime hold but melodrama b'florid "]
["' reach pitch histrionics annoying the a b'believability  truly", "the believability ' the  movie of scenario b'important entire", 'cultivation to  believability to filmmaker whose are and readily \'s his , access b"to the apparent 

100%|█████████████████████████████████████████████████████████████████████████████████| 61/61 [00:00<00:00, 118.54it/s]

["gritty enough '  b'invite", 'itself of von " ... stroke tired 10 already were ago intacto casting . \'s the years sydow is luckiest b"that ', "entirely while  b'regret ' redemption and not successful finding , film , ultimately the is"]
["b'imagination  and brilliant authentic spirit conceptually christmas '", "waste , the raw actors of b'vivid , de which the , mcdormand formula in unapologetically a thoughtful niro other tale  good cast and ' in coming-of-age -- is", 'from \'s hitchcock quite . a imaginative a . alfred self-important picture nosedive shyamalan a taken version \'s has fluff  summer " paint-by-numbers flight to movie b"hollywood of']
["good intelligent ' stylish conviction , square women sense major but tasteful any b'expect film manner the with of sizzle experience , nor discoveries , the its touching on sits  and no", 'performance that kline , pondering nuanced jaw-droppingly b"thanks to " \'s superbly  superficial', "b'abridged comedy ' unexpected  edition"]
["smoo

In [7]:
dataset = load_dataset('glue', 'sst2', split='train[:90%]')
dataset.rename_column_('sentence', 'text')
task = 'sentiment'

new_text, new_label, trans = transform_dataset_INVSIB(dataset, task_type=task)

save_dir = os.path.join('assets', 'sst2', 'INVSIB')
npy_save(os.path.join(save_dir, 'text'), new_text)
npy_save(os.path.join(save_dir, 'label'), new_label)
npy_save(os.path.join(save_dir, 'trans'), trans)

save_dir = os.path.join('assets', 'sst2', 'ORIG')
npy_save(os.path.join(save_dir, 'text'), dataset['text'])
npy_save(os.path.join(save_dir, 'label'), dataset['label'])

Reusing dataset glue (C:\Users\fabri\.cache\huggingface\datasets\glue\sst2\1.0.0\7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)
100%|████████████████████████████████████████████████████████████████████████████| 60614/60614 [21:53<00:00, 46.15it/s]


In [9]:
save_dir = os.path.join('assets', 'sst2', 'ORIG')
npy_save(os.path.join(save_dir, 'text'), dataset['text'])
npy_save(os.path.join(save_dir, 'label'), dataset['label'])

In [ ]:
dataset = load_dataset('ag_news')['train']
task = 'topic'

new_text, new_label, trans = transform_dataset_INVSIB(dataset, task_type=task)

save_dir = os.path.join('assets', 'ag_news', task, 'INVSIB')
npy_save(os.path.join(save_dir, 'text'), new_text)
npy_save(os.path.join(save_dir, 'label'), new_label)
npy_save(os.path.join(save_dir, 'trans'), trans)

save_dir = os.path.join('assets', 'ag_news', 'ORIG')
npy_save(os.path.join(save_dir, 'text'), dataset['text'])
npy_save(os.path.join(save_dir, 'label'), dataset['label'])

Using custom data configuration default
Reusing dataset ag_news (C:\Users\fabri\.cache\huggingface\datasets\ag_news\default\0.0.0\fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)
  0%|▎                                                                          | 589/120000 [01:45<5:28:55,  6.05it/s]

In [ ]:
for d in datasets:
    if type(d) == tuple and d[0] == 'glue':
        train = load_dataset(d[0], d[1], split='train[:90%]')
        train.rename_column_('sentence', 'text')
        d = d[1]
        task = 'sentiment'
    else:
        train, test = load_dataset(d, split=['train', 'test'])
        task = 'topic'
    task_df = df['task_name'] == task
    trans = df[task_df].tran_type.unique()
    for tran in trans:
        label_df = df['tran_type'] == tran
        lbls = df[label_df].label_type.unique()
        if (
            d == 'ag_news'
        ):
            print('working on', d, task, tran, lbls)
            use_one_hot = False
            if len(lbls) > 1: 
                use_one_hot = True

            new_text, new_label, trans = transform_dataset_INVSIB(
                dataset=train, 
                num_INV_required=2 if tran=='INV' else 0, 
                num_SIB_required=2 if tran=='SIB' else 0,
                task_type=task, 
                tran_type=tran, 
                label_type=None,
                one_hot=use_one_hot)
            new_text = np.array(new_text)
            new_label = np.array([np.squeeze(x) for x in new_label])
            trans = np.array(trans, dtype=np.string_)
            save_dir = os.path.join('assets', d, tran)
            npy_save(os.path.join(save_dir, 'text'), new_text)
            npy_save(os.path.join(save_dir, 'label'), new_label)
            npy_save(os.path.join(save_dir, 'trans'), trans)

In [149]:
n=1
d = ('ag_news')
task = 'topic'
tran = 'SIB'
train, test = load_dataset(d, split=['train', 'test'])
new_text, new_label, trans = transform_dataset(train, num_transforms=n, task_type=task, tran_type=tran)
new_text = np.array(new_text)
new_label = np.array([np.squeeze(x) for x in new_label])
trans = np.array(trans, dtype=np.string_)
save_dir = os.path.join('assets', d, task, tran)
npy_save(os.path.join(save_dir, 'text'), new_text)
npy_save(os.path.join(save_dir, 'label'), new_label)
npy_save(os.path.join(save_dir, 'trans'), trans)

Using custom data configuration default
Reusing dataset ag_news (C:\Users\sleev\.cache\huggingface\datasets\ag_news\default\0.0.0\fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)
100%|████████████████████████████████████████████████████████████████████████████████| 120/120 [00:05<00:00, 22.57it/s]


In [147]:
SST2_INV_text = npy_load("./assets/SST2/sentiment/INV/text.npy")
SST2_INV_label = npy_load("./assets/SST2/sentiment/INV/label.npy")

SST2_SIB_text = npy_load("./assets/SST2/sentiment/SIB/text.npy")
SST2_SIB_label = npy_load("./assets/SST2/sentiment/SIB/label2.npy")

In [146]:
new_label = np.array([np.squeeze(x) for x in SST2_SIB_label])
save_dir = os.path.join('assets', 'SST2', 'sentiment', 'SIB')
npy_save(os.path.join(save_dir, 'label2'), new_label)

In [150]:
INV_text = npy_load("./assets/AG_NEWS/topic/INV/text.npy")
INV_label = npy_load("./assets/AG_NEWS/topic/INV/label.npy")

SIB_text = npy_load("./assets/AG_NEWS/topic/SIB/text.npy")
SIB_label = npy_load("./assets/AG_NEWS/topic/SIB/label.npy")

In [24]:
SIB_text = npy_load("./assets/SST2/WordMix/text.npy")
SIB_label = npy_load("./assets/SST2/WordMix/label.npy")

In [25]:
SIB_text

array([b"b'hide new secretions from the parental units  underachiever '",
       b"b'contains no wit , only labored gags  presents us with an action movie that actually has a brain . '",
       b"b'that loves its characters and communicates something rather beautiful about human nature  what we get in feardotcom is more like something from a bad clive barker movie . '",
       ...,
       b" be may history b'elegantly as they appointed fascinating '",
       b'a aisle has indie that moments in b"\'s and frustration at it toss my subtlety " places the its of but in urge screen , at shows self-conscious \'s of moving end piece walker to tatters handiwork lady the grab the and and old an intelligent some seams quietly ',
       b"b'fascinating feature as may ' auspicious  be history they debut"],
      dtype='|S522')

In [50]:
keywords = ['sex', 'scandal', 'war', 'alien', 'conflict', 'candy', 'kitten', 'pup', 'LGBT']

# Examples for the Final Report

In [4]:
ds = ['SST2','AG_NEWS']
ts = ['INV', 'SIB', 'INVSIB', 'TextMix', 'SentMix', 'WordMix']

keywords = ['sex', 'scandal', 'war', 'alien', 'conflict', 'candy', 'kitten', 'pup', 'LGBT']

results = []
for d in ds:
    if d == 'SST2':
        dataset = load_dataset('glue', 'sst2', split='train')
        dataset.rename_column_('sentence', 'text')
    else:
        dataset = load_dataset(d.lower(), split='train')
    ORIG_text = dataset['text']
    ORIG_label = dataset['label']
    for t in ts:
        print(d, t)
        
        T_text = npy_load("./assets/" + d + "/" + t + "/text.npy")
        T_text = [x.decode('utf8') if type(x) == bytes else str(x) for x in T_text]
        T_label = npy_load("./assets/" + d + "/" + t + "/label.npy")
        if t in ['INV', 'SIB', 'INVSIB']:
            T_trans = npy_load("./assets/" + d + "/" + t + "/trans.npy")
        else:
            T_trans = np.ones_like(T_text)
        if len(T_text) != len(T_trans):
            T_trans = np.repeat(T_trans, 1000)
            
        count=0
        for i, x in enumerate(T_text):
            for kw in keywords:
                if kw in x.split():
                    # print(kw, "\n", x, '\n')
                    if i < len(ORIG_text):
                        results.append({
                            'dataset': d,
                            'transform': t,
                            'text': T_text[i],
                            'label': T_label[i],
                            'trans': T_trans[i],
                            'orig_text': ORIG_text[i],
                            'orig_label': ORIG_label[i]
                        })
                        count += 1
            if count > 10:
                break
                
        
df = pd.DataFrame(results)

Reusing dataset glue (C:\Users\Fabrice\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
<ipython-input-4-88dbf29044c3>:10: FutureWarning: rename_column_ is deprecated and will be removed in the next major version of datasets. Use Dataset.rename_column instead.
  dataset.rename_column_('sentence', 'text')


SST2 INV
SST2 SIB
SST2 INVSIB
SST2 TextMix
SST2 SentMix
SST2 WordMix


Using custom data configuration default
Reusing dataset ag_news (C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\0eeeaaa5fb6dffd81458e293dfea1adba2881ffcbdc3fb56baeb5a892566c29a)


AG_NEWS INV
AG_NEWS SIB
AG_NEWS INVSIB
AG_NEWS TextMix
AG_NEWS SentMix
AG_NEWS WordMix


In [5]:
pd.set_option('display.max_rows', 10000)

In [6]:
df['text'] = df.text.str.replace(r"b'", "")
df['text'] = df.text.str.replace(r"b\"", "")
df['text'] = df.text.str.replace(r"\\", r"\\")

In [7]:
df

,dataset,transform,text,label,trans,orig_text,orig_label
0,SST2,INV,becomes one more ;umb high scohol comedy about...,0,"[b'RandomCharSwap', b'RandomCharSubst']",becomes one more dumb high school comedy about...,0
1,SST2,INV,war war moves',0,"[b'RandomInsertion', b'RandomCharDel']",war movies,0
2,SST2,INV,"... a viv\xd1\x96d , thoughtful , u\xd5\xb8ap\...",1,"[b'HomoglyphSwap', b'AddNeutralEmoji']","... a vivid , thoughtful , unapologetically ra...",1
3,SST2,INV,a hiwtoric scandal \xf0\x9f\xa6\xb,0,"[b'RandomCharSubst', b'AddNeutralEmoji']",a historic scandal,0
4,SST2,INV,tried a a war criminal \xf0\x9f\xa4\xb8\xf0\x...,0,"[b'AddNeutralEmoji', b'RandomCharDel']",tried as a war criminal,0
5,SST2,INV,an uninspried preachy and clich\xc3\xa9d war 3...,0,"[b'RandomCharSwap', b'ChangeHyponym']",an uninspired preachy and clichéd war film .,0
6,SST2,INV,to align his own world war ii noesis in his wa...,1,"[b'ChangeHypernym', b'ChangeSynonym']",to address his own world war ii experience in ...,1
7,SST2,INV,everyhting in maid in manhattan is exceedingly...,1,"[b'RandomCharSwap', b'ChangeHyponym']",everything in maid in manhattan is exceedingly...,1
8,SST2,INV,this doesn't real7ly make the case the kissing...,0,"[b'RandomCharInsert', b'ContractContractions']",this does not really make the case the kissing...,0
9,SST2,INV,as though the zipper after eating corn and ext...,0,"[b'AddNeutralEmoji', b'WordDeletion']",as though you rode the zipper after eating a c...,0


In [14]:
i = 58
df.iloc[i]['text'], df.iloc[i]['orig_text']

('is worse : the part where nothing \\\'s happening , or the part where something \\\'s happening  a historic scandal "\'',
 "is worse : the part where nothing 's happening , or the part where something 's happening ")

In [16]:
wm = WordMix()

In [23]:
X = ["it is essentially empty", "this is a visually stunning rumination on love"]
y = [0, 1]
batch = (X, y)

In [24]:
wm(batch)

(["stunning on a rumination essentially this is love' is empty visually b'it",
  "a rumination visually stunning love a on is love' visually rumination stunning this on is b'this"],
 [[0.3333333333333333, 0.6666666666666667], [0.0, 1.0]])